In [1]:
from elasticsearch import Elasticsearch, helpers
from pandas import DataFrame

In [2]:
# returns a ES connection object
def connect_to_elastic():
    es = Elasticsearch([{'host':'localhost', 'port':9200, 'scheme': 'http'}], verify_certs=True)
    try:
        if es.ping(): return es
    except ex:
        raise ValueError('Connection failed')

In [3]:
# matches data with default similarity
def get_data_default(keyword, index_name='books'):
    # establish connection
    es = connect_to_elastic()
    
    # elasticsearch query to match keyword on field title
    multi_match_query =  match_query = {
                                            "match": {
                                            "book_title": keyword
                                            }
                                        }
    
    # execute query and return matches
    res = es.search(index=index_name, query=multi_match_query, size=100)

    books = []
    score = []
    for hit in res['hits']['hits']:
        books.append(hit['_source'])
        score.append(hit['_score'])

    matched_books = DataFrame(books)
    matched_books['score'] = score

    return matched_books

In [4]:
get_data_default('computer').head()

,isbn,book_title,book_author,year_of_publication,publisher,summary,category,score
0,1565920864,Computer Crime (Computer Security),David Icove,1995,O'Reilly,"Aimed at those who need to understand, investi...",['computers'],10.461483
1,013001687X,Computer Simplified (MaranGraphics' simplified...,Richard Maran,2004,John Wiley & Sons Inc,"The most illustrated guide to PCs available, t...",['microcomputers'],9.533649
2,0132903393,Computer Ethics,Deborah G. Johnson,1993,Prentice Hall,"Written in clear, accessible prose, this text ...",['computers'],9.467008
3,0684854104,ZEN COMPUTER,Phil Toshio sudo,2001,Simon & Schuster,A philosophical blending of the principles of ...,['philosophy'],9.467008
4,0684854090,Zen Computer,Philip Toshio Sudo,1999,Simon & Schuster,The author of &quot;Zen Guitar&quot; once more...,['self-help'],9.467008


In [7]:
def get_data_custom(keyword, user_id, index_name='books_avg_ratings'):
    
    # first match
    books = get_data_default(keyword, index_name)
    
    # isbns to find user's ratings
    isbns = ' '.join(books.loc[:, 'isbn'].values)
    
    # query
    match_isbn_query = {
            "bool": {
                "must": [{"match": {"isbn": isbns}}, {"match": {"uid": user_id}}]
                }
        }
    
    
    # reconnect
    es = connect_to_elastic()
    
    # execute query and return matches
    res = es.search(index='books_ratings', query=match_isbn_query)
    
    # create dictionary with isbn : rating paris
    user_ratings = {}
    for hit in res['hits']['hits']:
        user_ratings[hit['_source']['isbn']] = float(hit['_source']['rating'])
    
    score = []
    # formulating and calculating the scores
    for i in range(len(books)):
        # get each isbn
        isbn = books.loc[i, 'isbn']
        # check if book is rated by user, put his rating else make 0
        ur = user_ratings[isbn] if isbn in user_ratings else 0
        # contribute to final scoring
        score += [(.8 * books.loc[i, 'score']) + (.1 * float(books.loc[i, 'rating'])) + .1 * ur]

    # replace with the new custom scores
    books['score'] = score

    # sort by 'score'
    return books.drop(columns='rating').sort_values(by=['score'], ascending=False)

In [8]:
get_data_custom('computer', 35859)

,isbn,book_title,summary,score
0,1565920864,Computer Crime (Computer Security),"Aimed at those who need to understand, investi...",8.835853
1,013001687X,Computer Simplified (MaranGraphics' simplified...,"The most illustrated guide to PCs available, t...",8.126919
3,0684854104,ZEN COMPUTER,A philosophical blending of the principles of ...,8.073606
6,0133499456,Computer Networks,The book takes a structured approach to networ...,7.923606
2,0132903393,Computer Ethics,"Written in clear, accessible prose, this text ...",7.573606
...,...,...,...,...
83,156711086X,Steven Jobs & Stephen Wozniak: Creating the Ap...,Profiles the inventors who developed and marke...,4.685207
93,0670812781,West of Eden: The End of Innocence at Apple Co...,Traces the growth of Apple Computer from a sma...,4.643134
90,155615223X,Out of the Inner Circle: The True Story of a C...,"Out of the Inner Circle is a compelling, first...",4.580241
96,1558604286,Computer Organization and Design Second Editio...,The performance of software systems is dramati...,4.443134
